In [1]:
import numpy as np
import pandas as pd
import tensorflow.keras as keras

In [2]:
from collections import Counter

# 基础数据加工

In [3]:
dt = pd.read_csv("../data/user2up.csv", encoding="utf_8_sig")
dt.rename(columns={"from": "user", "mid": "item"}, inplace=True)
dt2 = dt[["user", "item"]]
dt2 = dt2.astype(int)
# dt2 = dt2[:10000]
# dt2 = dt2.sample(n=1000, random_state=123)

matrix_user2up = dt2.groupby("user").agg(list)

# items = {key: value for value, key in enumerate(set(dt2["item"]))}
items = set(dt2["item"])

In [4]:
users = set(dt2.user)
items = set(dt2.item)

In [5]:
def onehot_spec(raw):
    raw = set(raw)
    return [1 if item in raw else 0 for item in items]


matrix_user2up["vec"] = matrix_user2up["item"].apply(onehot_spec)

dt3 = matrix_user2up["vec"].sample(frac=1, random_state=123).values
dt3 = np.array([np.array(i) for i in dt3])

split_n = int(len(dt3) * 0.7)
X_train = dt3[:split_n]
X_test = dt3[split_n:]

In [6]:
dims = len(X_train[0])

# sklearn的KMeans(仅支持欧氏距离)

In [7]:
from sklearn.cluster import KMeans

In [8]:
X_train.shape

(2669, 96167)

In [9]:
model = KMeans(n_clusters=4)

In [10]:
model.fit(dt3)

KMeans(n_clusters=4)

In [11]:
ys_sklearn_eu = model.predict(dt3)

In [12]:
ct_sklearn_ud = Counter(ys_sklearn_eu)

In [13]:
ct_sklearn_ud

Counter({3: 3369, 0: 441, 1: 3, 2: 1})

#### 欧氏距离对于这种高维稀疏的向量聚类效果并不好 

# NLTK的KMeans(支持欧式距离/余弦距离/自定义距离)

In [14]:
import nltk

In [15]:
from nltk.cluster.kmeans import KMeansClusterer

In [16]:
from nltk.cluster.util import euclidean_distance

In [17]:
model = KMeansClusterer(num_means=20,distance=nltk.cluster.util.cosine_distance)
model.cluster(dt3[:])

In [19]:
ys_NLTK_cos = [model.classify(i) for i in dt3]

In [20]:
ct_NLTK_cos = Counter(ys_NLTK_cos)

In [21]:
ct_NLTK_cos

Counter({14: 112,
         4: 274,
         19: 616,
         15: 258,
         9: 447,
         0: 370,
         13: 166,
         5: 61,
         1: 294,
         2: 203,
         3: 141,
         17: 62,
         6: 133,
         8: 56,
         11: 199,
         16: 124,
         7: 46,
         18: 194,
         10: 31,
         12: 27})

In [34]:
matrix_user2up["vec_class"] = ys_NLTK_cos

In [35]:
matrix_user2up

,item,vec,vec_class
user,,,
1,"[335115, 2, 12, 9099524, 867152]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",14
2,"[1868902080, 623512391, 43855, 520811440, 3941...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, ...",4
4,"[9617619, 7584632, 2026561407, 22721144, 48163...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, ...",19
6,"[454560170, 488034462, 102774573, 346563107, 1...","[0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, ...",15
8,"[319274146, 1724598, 389552703, 489412051, 142...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",9
...,...,...,...
1999942692,"[174501086, 320840078, 473222648, 37090048]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2013795079,"[393275307, 360429426, 6831706, 20912712, 2659...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",18
2016676980,"[386917389, 473222648, 37090048]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2


In [36]:
KMeansClusterer?

## 尝试jaccard距离效果

In [40]:
def jaccard(u,v):
    u = [1 if i else 0 for i in u]
    v = [1 if i else 0 for i in v]
    tmp = len([1 for i,j in zip(u,v) if i+j==2 ])
    return tmp/(sum(u)+sum(v)-tmp)
model = KMeansClusterer(num_means=8,distance=jaccard)

In [ ]:
model.cluster(dt3[:])